In [1]:
%load_ext autoreload
%autoreload 2

In [51]:
from sklearn.tree import DecisionTreeRegressor
import pandas as pd
import numpy as np
#from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import cross_val_score, KFold
from sklearn.model_selection import train_test_split
#from sklearn.model_selection import train_test_split
from utils import *

In [4]:
dataframes = read_csv("./datasets", 'csv')

In [6]:
res = prepare_dataframes(dataframes)

  0%|          | 0/3 [00:00<?, ?it/s]

out_hum
pressure
out_temp


In [7]:
df = get_labels(res, '2024-02-13')

In [8]:
df['rain_1h'] = df['rain_1h'].fillna(0)
df['snow_1h'] = df['snow_1h'].fillna(0)
df['total percipitation'] = df['rain_1h'] + df['snow_1h']

In [9]:
df['last_changed_count'] = df.groupby('last_changed')['last_changed'].transform('count')
df = df.drop_duplicates(subset=['last_changed'])
df = df.loc[:, ['last_changed', 'pressure', 'out_temp', 'out_hum', 'total percipitation']]
df = df.reset_index(drop=True)

## Tabela 1 (time, P, T, H, R)

In [10]:
df

,last_changed,pressure,out_temp,out_hum,total percipitation
0,2023-09-20 00:00:00+00:00,986.000000,15.300000,76.000000,0.00
1,2023-09-20 01:00:00+00:00,985.428571,14.528571,78.900000,0.00
2,2023-09-20 02:00:00+00:00,985.760000,14.212500,79.500000,0.00
3,2023-09-20 03:00:00+00:00,985.280000,14.041667,81.300000,0.00
4,2023-09-20 04:00:00+00:00,985.666667,13.833333,83.333333,0.25
...,...,...,...,...,...
3499,2024-02-12 19:00:00+00:00,974.777778,7.220000,85.166667,0.00
3500,2024-02-12 20:00:00+00:00,975.846154,7.020000,84.933333,0.00
3501,2024-02-12 21:00:00+00:00,976.846154,6.566667,84.033333,0.00
3502,2024-02-12 22:00:00+00:00,976.521739,6.370000,83.873913,0.00


In [11]:
df.value_counts('total percipitation')

total percipitation
0.00     2666
0.25      166
0.51      149
0.38       37
1.02       29
         ... 
1.53        1
1.59        1
1.61        1
1.66        1
30.99       1
Name: count, Length: 119, dtype: int64

## Tabela 2 (time, 0, -1, -2, -3...)

In [12]:
df2 = generate_data(df, 5)
df2

,time,TP t+1,Press t0,Temp t0,Hum t0,TP t0,Press t-1,Temp t-1,Hum t-1,TP t-1,...,Hum t-3,TP t-3,Press t-4,Temp t-4,Hum t-4,TP t-4,Press t-5,Temp t-5,Hum t-5,TP t-5
0,2023-09-20 05:00:00+00:00,0.0,986.217391,13.966667,83.366667,0.0,985.666667,13.833333,83.333333,0.25,...,79.500000,0.00,985.428571,14.528571,78.900000,0.00,986.000000,15.300000,76.000000,0.00
1,2023-09-20 06:00:00+00:00,0.0,986.478261,14.640000,80.100000,0.0,986.217391,13.966667,83.366667,0.00,...,81.300000,0.00,985.760000,14.212500,79.500000,0.00,985.428571,14.528571,78.900000,0.00
2,2023-09-20 07:00:00+00:00,0.0,986.739130,15.480000,75.420000,0.0,986.478261,14.640000,80.100000,0.00,...,83.333333,0.25,985.280000,14.041667,81.300000,0.00,985.760000,14.212500,79.500000,0.00
3,2023-09-20 08:00:00+00:00,0.0,986.090909,17.000000,69.840000,0.0,986.739130,15.480000,75.420000,0.00,...,83.366667,0.00,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,81.300000,0.00
4,2023-09-20 09:00:00+00:00,0.0,986.200000,18.600000,61.920000,0.0,986.090909,17.000000,69.840000,0.00,...,80.100000,0.00,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,83.333333,0.25
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,2024-02-12 18:00:00+00:00,0.0,974.642857,7.371429,84.830000,0.0,973.500000,7.540000,84.800000,0.00,...,83.640000,0.00,971.652174,7.860000,85.928571,0.00,971.130435,7.955556,86.000000,0.00
3494,2024-02-12 19:00:00+00:00,0.0,974.777778,7.220000,85.166667,0.0,974.642857,7.371429,84.830000,0.00,...,83.020000,0.00,972.384615,7.877778,83.640000,0.00,971.652174,7.860000,85.928571,0.00
3495,2024-02-12 20:00:00+00:00,0.0,975.846154,7.020000,84.933333,0.0,974.777778,7.220000,85.166667,0.00,...,84.800000,0.00,973.222222,7.754545,83.020000,0.00,972.384615,7.877778,83.640000,0.00
3496,2024-02-12 21:00:00+00:00,0.0,976.846154,6.566667,84.033333,0.0,975.846154,7.020000,84.933333,0.00,...,84.830000,0.00,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,83.020000,0.00


# DT (bez normalizacji danych)

In [13]:
X = df2.iloc[:,2:-1]
y = df2.iloc[:,1:2]

In [14]:
X

,Press t0,Temp t0,Hum t0,TP t0,Press t-1,Temp t-1,Hum t-1,TP t-1,Press t-2,Temp t-2,...,Temp t-3,Hum t-3,TP t-3,Press t-4,Temp t-4,Hum t-4,TP t-4,Press t-5,Temp t-5,Hum t-5
0,986.217391,13.966667,83.366667,0.0,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,...,14.212500,79.500000,0.00,985.428571,14.528571,78.900000,0.00,986.000000,15.300000,76.000000
1,986.478261,14.640000,80.100000,0.0,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,...,14.041667,81.300000,0.00,985.760000,14.212500,79.500000,0.00,985.428571,14.528571,78.900000
2,986.739130,15.480000,75.420000,0.0,986.478261,14.640000,80.100000,0.00,986.217391,13.966667,...,13.833333,83.333333,0.25,985.280000,14.041667,81.300000,0.00,985.760000,14.212500,79.500000
3,986.090909,17.000000,69.840000,0.0,986.739130,15.480000,75.420000,0.00,986.478261,14.640000,...,13.966667,83.366667,0.00,985.666667,13.833333,83.333333,0.25,985.280000,14.041667,81.300000
4,986.200000,18.600000,61.920000,0.0,986.090909,17.000000,69.840000,0.00,986.739130,15.480000,...,14.640000,80.100000,0.00,986.217391,13.966667,83.366667,0.00,985.666667,13.833333,83.333333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3493,974.642857,7.371429,84.830000,0.0,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,...,7.877778,83.640000,0.00,971.652174,7.860000,85.928571,0.00,971.130435,7.955556,86.000000
3494,974.777778,7.220000,85.166667,0.0,974.642857,7.371429,84.830000,0.00,973.500000,7.540000,...,7.754545,83.020000,0.00,972.384615,7.877778,83.640000,0.00,971.652174,7.860000,85.928571
3495,975.846154,7.020000,84.933333,0.0,974.777778,7.220000,85.166667,0.00,974.642857,7.371429,...,7.540000,84.800000,0.00,973.222222,7.754545,83.020000,0.00,972.384615,7.877778,83.640000
3496,976.846154,6.566667,84.033333,0.0,975.846154,7.020000,84.933333,0.00,974.777778,7.220000,...,7.371429,84.830000,0.00,973.500000,7.540000,84.800000,0.00,973.222222,7.754545,83.020000


In [15]:
y

,TP t+1
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0
...,...
3493,0.0
3494,0.0
3495,0.0
3496,0.0


## Test 1: DT, min_samples_leaf

In [26]:
kfold = KFold(n_splits = 5)

classifiers = [    
    DecisionTreeRegressor(min_samples_leaf=10),
    DecisionTreeRegressor(min_samples_leaf=20),
    DecisionTreeRegressor(min_samples_leaf=50),
    DecisionTreeRegressor(min_samples_leaf=100),
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        #'Degree': classifier.get_params().get('degree'),
        #'C': classifier.get_params().get('C'),
        #'Params': classifier.get_params(),
        #**classifier.get_params(),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_dt_1 = pd.DataFrame(results_list)

  0%|          | 0/4 [00:00<?, ?it/s]

In [27]:
df_dt_1

,Classifier,min_samples_leaf,r2,neg_mean_squared_error,neg_mean_absolute_error
0,DecisionTreeRegressor,10,-0.583974,-1.040221,-0.274358
1,DecisionTreeRegressor,20,-0.181690,-0.967274,-0.247211
2,DecisionTreeRegressor,50,-0.012621,-0.927437,-0.226583
3,DecisionTreeRegressor,100,0.073435,-0.905046,-0.242079


## Test 2: DT, min_samples_leaf

In [31]:
kfold = KFold(n_splits = 5)

classifiers = [    
    DecisionTreeRegressor(min_samples_leaf=1000),
    DecisionTreeRegressor(min_samples_leaf=10000),
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        #'Degree': classifier.get_params().get('degree'),
        #'C': classifier.get_params().get('C'),
        #'Params': classifier.get_params(),
        #**classifier.get_params(),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_dt_2 = pd.DataFrame(results_list)

  0%|          | 0/2 [00:00<?, ?it/s]

In [32]:
df_dt_2

,Classifier,min_samples_leaf,r2,neg_mean_squared_error,neg_mean_absolute_error
0,DecisionTreeRegressor,1000,-0.001848,-0.968791,-0.283691
1,DecisionTreeRegressor,10000,-0.035639,-0.972458,-0.297085


## Test 3: DT, min_samples_leaf

In [35]:
kfold = KFold(n_splits = 5)

classifiers = [    
    DecisionTreeRegressor(min_samples_leaf=100),
    DecisionTreeRegressor(min_samples_leaf=200),
    DecisionTreeRegressor(min_samples_leaf=300),
    DecisionTreeRegressor(min_samples_leaf=400),
    DecisionTreeRegressor(min_samples_leaf=500),
    DecisionTreeRegressor(min_samples_leaf=600),
    DecisionTreeRegressor(min_samples_leaf=700),
    DecisionTreeRegressor(min_samples_leaf=800),
    DecisionTreeRegressor(min_samples_leaf=900),
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        #'Degree': classifier.get_params().get('degree'),
        #'C': classifier.get_params().get('C'),
        #'Params': classifier.get_params(),
        #**classifier.get_params(),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_dt_3 = pd.DataFrame(results_list)

  0%|          | 0/9 [00:00<?, ?it/s]

In [37]:
df_dt_3 #600 bo R2 i 100

,Classifier,min_samples_leaf,r2,neg_mean_squared_error,neg_mean_absolute_error
0,DecisionTreeRegressor,100,0.073436,-0.905070,-0.242151
1,DecisionTreeRegressor,200,0.047001,-0.923796,-0.239486
2,DecisionTreeRegressor,300,0.081076,-0.918612,-0.239280
3,DecisionTreeRegressor,400,0.079199,-0.918227,-0.234359
4,DecisionTreeRegressor,500,0.085761,-0.917369,-0.229830
5,DecisionTreeRegressor,600,0.102188,-0.915017,-0.225672
6,DecisionTreeRegressor,700,0.038093,-0.965705,-0.278675
7,DecisionTreeRegressor,800,-0.003432,-0.971110,-0.284707
8,DecisionTreeRegressor,900,-0.002240,-0.968999,-0.284250


## Test 4: DT, min_samples_leaf, min_samples_split

In [39]:
kfold = KFold(n_splits = 5)

classifiers = [    
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=2),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=10),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=50),
    
    DecisionTreeRegressor(min_samples_leaf=600, min_samples_split=2),
    DecisionTreeRegressor(min_samples_leaf=600, min_samples_split=10),
    DecisionTreeRegressor(min_samples_leaf=600, min_samples_split=50),

]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_dt_4 = pd.DataFrame(results_list)

  0%|          | 0/6 [00:00<?, ?it/s]

In [40]:
df_dt_4

,Classifier,min_samples_leaf,min_samples_split,r2,neg_mean_squared_error,neg_mean_absolute_error
0,DecisionTreeRegressor,100,2,0.073174,-0.905071,-0.242041
1,DecisionTreeRegressor,100,10,0.073171,-0.905071,-0.242073
2,DecisionTreeRegressor,100,50,0.073447,-0.905046,-0.242080
3,DecisionTreeRegressor,600,2,0.102188,-0.915017,-0.225672
4,DecisionTreeRegressor,600,10,0.102188,-0.915017,-0.225672
5,DecisionTreeRegressor,600,50,0.102188,-0.915014,-0.225672


## Test 5: DT, min_samples_leaf, min_samples_split

In [43]:
kfold = KFold(n_splits = 5)

classifiers = [    
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=100),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=500),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=600),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=700),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=800),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=900),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=1000),
    
    DecisionTreeRegressor(min_samples_leaf=600, min_samples_split=100),
    DecisionTreeRegressor(min_samples_leaf=600, min_samples_split=500),
    DecisionTreeRegressor(min_samples_leaf=600, min_samples_split=600),
    DecisionTreeRegressor(min_samples_leaf=600, min_samples_split=700),
    DecisionTreeRegressor(min_samples_leaf=600, min_samples_split=800),
    DecisionTreeRegressor(min_samples_leaf=600, min_samples_split=900),
    DecisionTreeRegressor(min_samples_leaf=600, min_samples_split=1000),  

]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_dt_5 = pd.DataFrame(results_list)

  0%|          | 0/14 [00:00<?, ?it/s]

In [44]:
df_dt_5 #mis_samples_leaf = 100, min_samples_split = 500

,Classifier,min_samples_leaf,min_samples_split,r2,neg_mean_squared_error,neg_mean_absolute_error
0,DecisionTreeRegressor,100,100,0.073446,-0.905070,-0.242140
1,DecisionTreeRegressor,100,500,0.096184,-0.899089,-0.233360
2,DecisionTreeRegressor,100,600,0.092680,-0.909465,-0.235235
3,DecisionTreeRegressor,100,700,0.092698,-0.909437,-0.234930
4,DecisionTreeRegressor,100,800,0.092700,-0.909436,-0.234935
5,DecisionTreeRegressor,100,900,0.092768,-0.909368,-0.235206
6,DecisionTreeRegressor,100,1000,0.092594,-0.909320,-0.235065
7,DecisionTreeRegressor,600,100,0.102170,-0.915017,-0.225672
8,DecisionTreeRegressor,600,500,0.102170,-0.915017,-0.225718
9,DecisionTreeRegressor,600,600,0.102188,-0.915017,-0.225672


## Test 6: DT, min_samples_leaf, min_samples_split

In [45]:
kfold = KFold(n_splits = 5)

classifiers = [    
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=400),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=450),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=500),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=550),
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_dt_6 = pd.DataFrame(results_list)

  0%|          | 0/4 [00:00<?, ?it/s]

In [46]:
df_dt_6

,Classifier,min_samples_leaf,min_samples_split,r2,neg_mean_squared_error,neg_mean_absolute_error
0,DecisionTreeRegressor,100,400,0.089992,-0.901562,-0.235197
1,DecisionTreeRegressor,100,450,0.094802,-0.899359,-0.232866
2,DecisionTreeRegressor,100,500,0.096173,-0.899089,-0.233360
3,DecisionTreeRegressor,100,550,0.096383,-0.899078,-0.233547


## Test 7: DT, min_samples_leaf, min_samples_split

In [47]:
kfold = KFold(n_splits = 5)

classifiers = [    
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=420),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=430),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=440),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=450),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=460),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=470),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=480),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=490),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=500),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=510),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=520),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=530),
    DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=540),
    
]

results_list = []
for classifier in tqdm(classifiers):
    #classifier.set_params(**{'C': 0.5})
    #print(type(classifier).__name__, classifier.get_params())

    result = {
        'Classifier': type(classifier).__name__, 
        'min_samples_leaf': classifier.get_params().get('min_samples_leaf'),
        'min_samples_split': classifier.get_params().get('min_samples_split'),
    }
    
    for scoring in ['r2', 'neg_mean_squared_error', 'neg_mean_absolute_error']:
        score = cross_val_score(classifier, X, y.values.ravel(), cv = kfold, scoring=scoring) 
        
        result[scoring] = score.mean()

    results_list.append(result)

df_dt_7 = pd.DataFrame(results_list)

  0%|          | 0/13 [00:00<?, ?it/s]

In [49]:
df_dt_7 #min_samples_leaf= 100, min_samples_spli = 530

,Classifier,min_samples_leaf,min_samples_split,r2,neg_mean_squared_error,neg_mean_absolute_error
0,DecisionTreeRegressor,100,420,0.090340,-0.901202,-0.234935
1,DecisionTreeRegressor,100,430,0.090239,-0.901204,-0.234967
2,DecisionTreeRegressor,100,440,0.094861,-0.899367,-0.232872
3,DecisionTreeRegressor,100,450,0.094861,-0.899368,-0.232889
4,DecisionTreeRegressor,100,460,0.094626,-0.899447,-0.233839
5,DecisionTreeRegressor,100,470,0.094626,-0.899457,-0.233839
6,DecisionTreeRegressor,100,480,0.094606,-0.899447,-0.233828
7,DecisionTreeRegressor,100,490,0.096309,-0.899072,-0.233405
8,DecisionTreeRegressor,100,500,0.096152,-0.899089,-0.233431
9,DecisionTreeRegressor,100,510,0.096173,-0.899099,-0.233360


## Test 8: DT, min_samples_leaf, min_samples_split

In [52]:
X_train, X_test, y_train, y_test = train_test_split(
    X,
    y,
    test_size=0.2,
    random_state=42
)

In [56]:
dtr = DecisionTreeRegressor(min_samples_leaf=100, min_samples_split=530)
dtr.fit(X_train, y_train.values.ravel())
y_pred = dtr.predict(X_test)

In [60]:
decision_path_matrix = dtr.decision_path(X_train, check_input=True)

In [61]:
print(decision_path_matrix)


  (0, 0)	1
  (0, 1)	1
  (0, 2)	1
  (0, 4)	1
  (0, 5)	1
  (0, 6)	1
  (0, 10)	1
  (0, 12)	1
  (0, 14)	1
  (0, 18)	1
  (1, 0)	1
  (1, 1)	1
  (1, 2)	1
  (1, 4)	1
  (1, 5)	1
  (1, 6)	1
  (1, 10)	1
  (1, 12)	1
  (1, 13)	1
  (2, 0)	1
  (2, 1)	1
  (2, 2)	1
  (2, 4)	1
  (2, 5)	1
  (2, 6)	1
  :	:
  (2794, 3)	1
  (2795, 0)	1
  (2795, 1)	1
  (2795, 2)	1
  (2795, 4)	1
  (2795, 5)	1
  (2795, 6)	1
  (2795, 10)	1
  (2795, 12)	1
  (2795, 13)	1
  (2796, 0)	1
  (2796, 1)	1
  (2796, 2)	1
  (2796, 4)	1
  (2796, 5)	1
  (2796, 6)	1
  (2796, 10)	1
  (2796, 12)	1
  (2796, 13)	1
  (2797, 0)	1
  (2797, 1)	1
  (2797, 2)	1
  (2797, 4)	1
  (2797, 5)	1
  (2797, 19)	1


In [63]:
dtr.get_depth()

10

In [64]:
dtr.get_n_leaves()

12

In [65]:
dtr.get_metadata_routing()

{'fit': {'sample_weight': None, 'check_input': None}, 'predict': {'check_input': None}, 'score': {'sample_weight': None}}